In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.applications import imagenet_utils
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
import itertools
import os
import shutil
import random
import glob
import matplotlib.pyplot as plt
%matplotlib inline

import sklearn
import pandas as pd
from sklearn.metrics import roc_auc_score, roc_curve, auc
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_score, recall_score
from scipy import interp
import matplotlib.pyplot as plt
import time

Using TensorFlow backend.
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\fatemeh\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 2963, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-1-d9c63dd8c5bf>", line 13, in <module>
    from sklearn.metrics import plot_confusion_matrix
ImportError: cannot import name 'plot_confusion_matrix'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\fatemeh\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 1863, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'ImportError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\fatemeh\Anaconda3\lib\site-packages\IPython\core\ultratb.py", line 1095, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "

ImportError: cannot import name 'plot_confusion_matrix'

In [2]:
#loading data
dataset = np.load('data.row/ASD2.npz') #Dataset ready in numpy array (removing background, resizing, and transforming into grayscale)
X = dataset['X']
y = dataset['y']

print(X.shape)
print(y.shape)

(3103, 224, 224, 3)
(3103, 1)


In [3]:
#Hyperparameters
nfolds = 3
nEpochs = 30
nBatch= 16
#inputDim = X.shape[1] # Count of features

In [4]:
kfold = StratifiedKFold(n_splits=nfolds, shuffle=True, random_state=1).split(X, y) #Cross validation
for train, test in kfold:
  print(X[train].shape, X[test].shape ,y[train].shape, y[test].shape )

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\fatemeh\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 2963, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-4-7ebc168295b6>", line 1, in <module>
    kfold = StratifiedKFold(n_splits=nfolds, shuffle=True, random_state=1).split(X, y) #Cross validation
NameError: name 'StratifiedKFold' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\fatemeh\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 1863, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\fatemeh\Anaconda3\lib\site-packages\IPython\core\ultratb.py", line 1095, in get_records
    return _fixed_getinnerframes(etb, number

NameError: name 'StratifiedKFold' is not defined

In [6]:
resnet50 = tf.keras.applications.ResNet50() 
#resnet50.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


102973440/102967424 [==============================] - 1700s 17us/step


In [8]:
x = resnet50.layers[-2].output

In [9]:
output = Dense(units=1, activation='sigmoid')(x)

In [10]:
model = Model(inputs=resnet50.input, outputs=output)

In [11]:
#for layer in model.layers[:-23]:
#   layer.trainable = False

for layer in model.layers[175:]:
    layer.trainable = True
for layer in model.layers[:175]:
    layer.trainable = False
#for i, layer in enumerate(model.layers):
    #print(i, layer.name, layer.trainable)

In [28]:
#from keras.optimizers import SGD
from keras.optimizers import Adam
#opt = SGD(lr=0.001,momentum=0.9,decay=0.01,nesterov=False)
opt = Adam(learning_rate=0.0001,beta_1=0.99,beta_2=0.99,epsilon=0.1,amsgrad=False,name="Adam",)
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
#model.summary()

In [ ]:
#Fiting the model 

hist =model.fit(X[train], y[train], validation_split=0.2, epochs=nEpochs, batch_size=nBatch, verbose=2)
  #pred = model.predict(X[test]).ravel()

  #Calculating Recall and precision
  #recall.append( recall_score(y[test], pred.round()) )
  #precision.append( precision_score(y[test], pred.round()) )

  #ROC AUC
  #fpr, tpr, thresholds = roc_curve(y[test], pred)
  #tprs.append(interp(mean_fpr, fpr, tpr))
  #tprs[-1][0] = 0.0
  #roc_auc = auc(fpr, tpr)
  #print(roc_auc)
  #aucs.append(roc_auc)
  #hists_CNN.append(hist)


#print("Avg AUC:", np.mean(aucs))

#print("Avg Recall:", np.mean(recall))
#print("Avg Precision:", np.mean(precision))



In [26]:
score = model.evaluate(X[test], y[test], verbose=0)
print(score)

[0.5941858291625977, 0.68858802318573]


In [27]:
y_pred = model.predict(X[test])

AUC = roc_auc_score(y[test], y_pred.round())
print(AUC)

0.6198308270676691
